In [97]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

In [283]:
# def users_to_json(spark_context, file):
#     def json_lambda(line):
#         json_line = {}
# #         tags_list={'Id':lambda x: int(x[1:-2]),
# #                    'Reputation':lambda x: int(x[1:-2]),
# #                    'CreationDate':lambda x: str(x)[1:-2],
# #                    'DisplayName':lambda x: str(x)[1:-2],
# #                    'LastAccessDate':lambda x: str(x)[1:-2],
# #                    'WebsiteUrl':lambda x: str(x)[1:-2],
# #                    'Location':lambda x: str(x)[1:-2],
# #                    'AboutMe':str,
# #                    'Views':lambda x: int(x[1:-2]),
# #                    'UpVotes':lambda x: int(x[1:-2]),
# #                    'DownVotes':lambda x: int(x[1:-2]),
# #                    'ProfileImageUrl':lambda x: str(x)[1:-2],
# #                    'EmailHash':lambda x: str(x)[1:-2],
# #                    'AccountId':lambda x: int(x[1:-2]),
# #                    'Age':lambda x: float(x[1:-2])}
#         tags_list={'Id':str,
#                    'Reputation':str,
#                    'CreationDate':str,
#                    'DisplayName':str,
#                    'LastAccessDate':str,
#                    'WebsiteUrl':str,
#                    'Location':str,
#                    'AboutMe':str,
#                    'Views':str,
#                    'UpVotes':str,
#                    'DownVotes':str,
#                    'ProfileImageUrl':str,
#                    'EmailHash':str,
#                    'AccountId':str,
#                    'Age':str}
#         indices = [line.find(tag,0) for tag in tags_list if line.find(tag,0)!=-1]
#         for i in range(len(indices)-1):
#             parsed = line[indices[i]:indices[i+1]].split('=')
#             try:
#                 json_line[parsed[0]] = tags_list[parsed[0]](parsed[1])
#             except ValueError:
#                 json_line[parsed[0]] = tags_list[parsed[0]](parsed[1][1:-2])
#         all_keys_json = {}
#         for tag in tags_list:
#             try:
#                 all_keys_json[tag] = json_line[tag]
#             except KeyError:
#                 all_keys_json[tag] = None
#         return all_keys_json
#     text = spark_context.textFile(file)
#     text = spark.sparkContext.textFile('Users.xml')
#     usersRDD = text.filter(filter_crap)
#     usersRDD = usersRDD.map(json_lambda)
#     return usersRDD



In [ ]:
def users_to_json(spark_context, file):
    def json_lambda(line):
        json_line = {}
        tags_list={'Id':str,
                   'Reputation':str,
                   'CreationDate':str,
                   'DisplayName':str,
                   'LastAccessDate':str,
                   'WebsiteUrl':str,
                   'Location':str,
                   'AboutMe':str,
                   'Views':str,
                   'UpVotes':str,
                   'DownVotes':str,
                   'ProfileImageUrl':str,
                   'EmailHash':str,
                   'AccountId':str,
                   'Age':str}
        indices = [line.find(tag,0) for tag in tags_list if line.find(tag,0)!=-1]
        for i in range(len(indices)-1):
            parsed = line[indices[i]:indices[i+1]].split('=')
            try:
                json_line[parsed[0]] = tags_list[parsed[0]](parsed[1])
            except ValueError:
                json_line[parsed[0]] = tags_list[parsed[0]](parsed[1][1:-2])
        all_keys_json = {}
        for tag in tags_list:
            try:
                all_keys_json[tag] = json_line[tag]
            except KeyError:
                all_keys_json[tag] = None
        return all_keys_json
    text = spark_context.textFile(file)
    text = spark.sparkContext.textFile('Users.xml')
    usersRDD = text.filter(filter_crap)
    usersRDD = usersRDD.map(json_lambda)
    return usersRDD


In [284]:
def filter_crap(s):
    if s.startswith("<?xml version=") or s=="<users>" or s=="</users>":
        return False
    else:
        return True
    
text = spark.sparkContext.textFile('Users.xml')
usersRDD = text.filter(filter_crap)

In [304]:
from xml.etree.ElementTree import fromstring
def sth(s):
    tree = fromstring(s)
    return tree.items()
new=usersRDD.map(sth)

In [308]:
from pyspark.sql.types import *
usersSchema = StructType([ \
    StructField("Id", StringType(), True), \
    StructField("Reputation", StringType(), True), \
    StructField("CreationDate", StringType(), True), \
    StructField("DisplayName", StringType(), True), \
    StructField("LastAccessDate", StringType(), True), \
    StructField("WebsiteUrl", StringType(), True), \
    StructField("Location", StringType(), True), \
    StructField("AboutMe", StringType(), True), \
    StructField("Views", StringType(), True), \
    StructField("UpVotes", StringType(), True),
    StructField("DownVotes", StringType(), True), \
    StructField("ProfileImageUrl", StringType(), True), \
    StructField("EmailHash", StringType(), True), \
    StructField("AccountId", StringType(), True), \
    StructField("Age", StringType(), True)])

In [313]:
df = new.map(lambda x: Row(**x))
#df = df.toDF(schema=usersSchema)

In [315]:
new.take(1)

[[('Id', '-1'),
  ('Reputation', '1'),
  ('CreationDate', '2016-01-11T22:16:50.167'),
  ('DisplayName', 'Community'),
  ('LastAccessDate', '2016-01-11T22:16:50.167'),
  ('Location', 'on the server farm'),
  ('AboutMe',
   '<p>Hi, I\'m not really a person.</p>\r\n<p>I\'m a background process that helps keep this site clean!</p>\r\n<p>I do things like</p>\r\n<ul>\r\n<li>Randomly poke old unanswered questions every hour so they get some attention</li>\r\n<li>Own community questions and answers so nobody gets unnecessary reputation from them</li>\r\n<li>Own downvotes on spam/evil posts that get permanently deleted</li>\r\n<li>Own suggested edits from anonymous users</li>\r\n<li><a href="http://meta.stackoverflow.com/a/92006">Remove abandoned questions</a></li>\r\n</ul>'),
  ('Views', '72'),
  ('UpVotes', '1'),
  ('DownVotes', '0'),
  ('AccountId', '-1')]]

In [276]:
a = users_to_json(spark.sparkContext, 'Users.xml')

In [277]:
from pyspark.sql.types import *
usersSchema = StructType([ \
    StructField("Id", StringType(), True), \
#     StructField("Reputation", StringType(), True), \
#     StructField("CreationDate", StringType(), True), \
#     StructField("DisplayName", StringType(), True), \
#     StructField("LastAccessDate", StringType(), True), \
#     StructField("WebsiteUrl", StringType(), True), \
#     StructField("Location", StringType(), True), \
#     StructField("AboutMe", StringType(), True), \
#     StructField("Views", StringType(), True), \
#     StructField("UpVotes", StringType(), True),
#     StructField("DownVotes", StringType(), True), \
#     #StructField("ProfileImageUrl", StringType(), True), \
#     StructField("EmailHash", StringType(), True), \
#     StructField("AccountId", StringType(), True), \
    StructField("Age", StringType(), True)])

In [278]:
df = a.map(lambda x: Row(**x))

In [279]:
df = df.toDF(schema=usersSchema)

In [280]:
df['Id']

Column<b'Id'>

In [282]:
df.take(10)

[Row(Id='"-1" ', Age=None),
 Row(Id='"1" ', Age=None),
 Row(Id='"2" ', Age=None),
 Row(Id='"3" ', Age=None),
 Row(Id='"4" ', Age=None),
 Row(Id='"5" ', Age=None),
 Row(Id='"7" ', Age=None),
 Row(Id='"9" ', Age=None),
 Row(Id='"10" ', Age=None),
 Row(Id='"11" ', Age=None)]